"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"

## Application

- planning, logistics and the manufacture of microchips, DNA sequencing => concept *city* represents customers, soldering points, or DNA fragments and the concept *distance* represents travelling cost or time, a similarity measure between DNA fragments.

- Another related problem: bottleneck travelling salesman problem -> Find a Hamiltonian cycle in a weighted graph with the minimal weight of the weightiest edge => real-world example: avoiding narrow streets with big buses.
    - important in transportation and logistics area. 
    - PCB manufacturing -> scheduling of a route of the drill machine to drill holes in a PCB.


## Solution 

- exact algorithm -> work reasonably fast only for small problem sizes
- heuristic algorithms -> deliver approximated solution in a reasonalbe time
- special cases for the problem for which either better or exact heuristics are possible.

### Exact Algorithm
- try all permutations to determine which one is cheapest (brute force search) -> O(n!)
- branch and bound algorithm -> process TSPs with thousands of nodes

### Heuristic Algorithm
- NN -> For N cities randomly distributed on a plane, the algorithm on average yields a path 25% longer than the shortest possible path
- The Algorithm of Christofides and Serdyukov

In [3]:
# Python3 program to solve 
# Traveling Salesman Problem using 
# Branch and Bound.
import math
maxsize = float('inf')

# Function to copy temporary solution
# to the final solution
def copyToFinal(curr_path):
	final_path[:N + 1] = curr_path[:]
	final_path[N] = curr_path[0]

# Function to find the minimum edge cost 
# having an end at the vertex i
def firstMin(adj, i):
	min = maxsize
	for k in range(N):
		if adj[i][k] < min and i != k:
			min = adj[i][k]

	return min

# function to find the second minimum edge 
# cost having an end at the vertex i
def secondMin(adj, i):
	first, second = maxsize, maxsize
	for j in range(N):
		if i == j:
			continue
		if adj[i][j] <= first:
			second = first
			first = adj[i][j]

		elif(adj[i][j] <= second and
			adj[i][j] != first):
			second = adj[i][j]

	return second

# function that takes as arguments:
# curr_bound -> lower bound of the root node
# curr_weight-> stores the weight of the path so far
# level-> current level while moving
# in the search space tree
# curr_path[] -> where the solution is being stored
# which would later be copied to final_path[]
def TSPRec(adj, curr_bound, curr_weight, 
			level, curr_path, visited):
	global final_res
	
	# base case is when we have reached level N 
	# which means we have covered all the nodes once
	if level == N:
		
		# check if there is an edge from
		# last vertex in path back to the first vertex
		if adj[curr_path[level - 1]][curr_path[0]] != 0:
			
			# curr_res has the total weight
			# of the solution we got
			curr_res = curr_weight + adj[curr_path[level - 1]]\
										[curr_path[0]]
			if curr_res < final_res:
				copyToFinal(curr_path)
				final_res = curr_res
		return

	# for any other level iterate for all vertices
	# to build the search space tree recursively
	for i in range(N):
		
		# Consider next vertex if it is not same 
		# (diagonal entry in adjacency matrix and 
		# not visited already)
		if (adj[curr_path[level-1]][i] != 0 and
							visited[i] == False):
			temp = curr_bound
			curr_weight += adj[curr_path[level - 1]][i]

			# different computation of curr_bound 
			# for level 2 from the other levels
			if level == 1:
				curr_bound -= ((firstMin(adj, curr_path[level - 1]) +
								firstMin(adj, i)) / 2)
			else:
				curr_bound -= ((secondMin(adj, curr_path[level - 1]) +
								firstMin(adj, i)) / 2)

			# curr_bound + curr_weight is the actual lower bound 
			# for the node that we have arrived on.
			# If current lower bound < final_res, 
			# we need to explore the node further
			if curr_bound + curr_weight < final_res:
				curr_path[level] = i
				visited[i] = True
				
				# call TSPRec for the next level
				TSPRec(adj, curr_bound, curr_weight, 
					level + 1, curr_path, visited)

			# Else we have to prune the node by resetting 
			# all changes to curr_weight and curr_bound
			curr_weight -= adj[curr_path[level - 1]][i]
			curr_bound = temp

			# Also reset the visited array
			visited = [False] * len(visited)
			for j in range(level):
				if curr_path[j] != -1:
					visited[curr_path[j]] = True

# This function sets up final_path
def TSP(adj):
	
	# Calculate initial lower bound for the root node 
	# using the formula 1/2 * (sum of first min + 
	# second min) for all edges. Also initialize the 
	# curr_path and visited array
	curr_bound = 0
	curr_path = [-1] * (N + 1)
	visited = [False] * N

	# Compute initial bound
	for i in range(N):
		curr_bound += (firstMin(adj, i) +
					secondMin(adj, i))

	# Rounding off the lower bound to an integer
	curr_bound = math.ceil(curr_bound / 2)

	# We start at vertex 1 so the first vertex 
	# in curr_path[] is 0
	visited[0] = True
	curr_path[0] = 0

	# Call to TSPRec for curr_weight 
	# equal to 0 and level 1
	TSPRec(adj, curr_bound, 0, 1, curr_path, visited)

# Driver code

# Adjacency matrix for the given graph
adj = [[0, 20, 30, 10,11],
	[15,0,16,4,2],
	[3,5,0,2,4],
	[19,6,18,0,3],
	[16,4,7,16,0]]
N = 5

# final_path[] stores the final solution 
# i.e. the // path of the salesman.
final_path = [None] * (N + 1)

# visited[] keeps track of the already
# visited nodes in a particular path
visited = [False] * N

# Stores the final minimum weight
# of shortest tour.
final_res = maxsize

TSP(adj)

print("Minimum cost :", final_res)
print("Path Taken : ", end = ' ')
for i in range(N + 1):
	print(final_path[i], end = ' ')



Minimum cost : 28
Path Taken :  0 3 1 4 2 0 

In [4]:
#sample adjacency matrix for the problem
dist = [[0, 20, 30, 10,11],
	[15,0,16,4,2],
	[3,5,0,2,4],
	[19,6,18,0,3],
	[16,4,7,16,0]]

- level -> represents the depth or the level of the node in the search tree. Keep track of how far we have traversed in the potential solution path.
- path -> a list that represents the sequence of cities visited so far in the current path.
- bound -> the lower bound cost associated with the current path.
- __lt__ -> compare nodes based on their `bound` attribute => used by Priority Queue to prirotize nodes based on thier bounds => nodes with the smallest bound will be explored first.
- lower bound cost -> heuristic estimate of the minimum possible cost to complete the tour from a given node.

**Calculation of the lower bound cost**
- basic lower bound cost using minimum edge costs -> for each unvisited node, add the minimum cost edge when leaving city to the bound.
- 1-tree relaxation -> construct a MST over the unvisited node and add the two smallest edges connecting the MST to the partial path.
- reduced cost matrix -> modify the original cost matrix to reduce the problem size while maintaining the same optimal tour cost, then calculate the bound using the reduced matrix

In [5]:
class Node(object):
    def __init__(self,level=None,path=None,bound=None) -> None:
        self.level = level
        self.path = path
        self.bound = bound
    
    def __lt__(self,other):
        return self.bound < other.bound

`bound` -> calculate the lower bound cost for a given partial path in the TSP.

In [33]:
from queue import PriorityQueue

def bound(dist,node):
    path = node.path # the current sequence of the cities visited.
    _bound = 0 

    # determine the remaining cities 
    
    n = len(dist) # the total number of cities
    last = path[-1] # all cities in the path except the last one and the last city in the current path
    remain = filter(lambda x: x not in path, range(n)) # list of cities that have not yet been visited

    #iterates over the current path and adds the cost of travelling from each city to the next in the sequence 
    #gives the acutal costs incurred so far along the visited cities forming part of the lower bound
    for i in range(len(path) - 1):
        _bound += dist[path[i]][path[i + 1]]

    # adds the minimum cost of travelling from the last city in the current path to any of the remaining cities.
    #ensures a plausible connection from the last city to the rest of the tour, 
    _bound += min([dist[last][i] for i in remain])

    #list of starting city and all remaining cities.
    #calculate the minimum outgoing edges for the remaining cities, ensuring that every city is considered for its future connection
    p = [path[0]] + [remain]

    # calculate minimum outgoing costs for remaining cities.
    #For each remaining city r, calculates the minimum cost to travel from r to any other city in the list p (excluding itself).
    for r in remain:
        _bound += min([dist[r][i] for i in filter(lambda x: x != r, p)])
    return _bound

In [16]:
def length(adj_mat, node):
    tour = node.path
    # returns the sum of two consecutive elements of tour in dist[i][j]
    return sum([adj_mat[tour[i]][tour[i + 1]] for i in range(len(tour) - 1)])

In [18]:
def travel(dist, src=0):
    optimal_tour = []
    n = len(dist)
    u = Node()
    PQ = PriorityQueue()
    optimal_length = 0
    v = Node(level=0, path=[0])
    min_length = float('inf')  # infinity
    v.bound = bound(dist, v)
    PQ.put(v)
    while not PQ.empty():
        v = PQ.get()
        if v.bound < min_length:
            u.level = v.level + 1
            for i in filter(lambda x: x not in v.path, range(1, n)):
                u.path = v.path[:]
                u.path.append(i)
                if u.level == n - 2:
                    l = set(range(1, n)) - set(u.path)
                    u.path.append(list(l)[0])
                    # putting the first vertex at last
                    u.path.append(0)

                    _len = length(dist, u)
                    if _len < min_length:
                        min_length = _len
                        optimal_length = _len
                        optimal_tour = u.path[:]

                else:
                    u.bound = bound(dist, u)
                    if u.bound < min_length:
                        PQ.put(u)
                # make a new node at each iteration! python it is!!
                u = Node(level=u.level)

    # shifting to proper source(start of path)
    optimal_tour_src = optimal_tour
    if src != 1:
        optimal_tour_src = optimal_tour[:-1]
        y = optimal_tour_src.index(src)
        optimal_tour_src = optimal_tour_src[y:] + optimal_tour_src[:y]
        optimal_tour_src.append(optimal_tour_src[0])

    return optimal_tour_src, optimal_length

In [34]:
print(travel(dist))

([0, 3, 1, 4, 2, 0], 28)
